In [ ]:
(import chemaxon.marvin.MolPrinter)
(import java.awt.Graphics2D)
(import java.awt.image.BufferedImage)
(import java.awt.Rectangle)

(def n (MolImporter/importMol "CN1C=NC2=C1C(=O)N(C)C(=O)N2C>>CN1C=NC2=C1C(=O)N(C)C(=O)N2C"))
  
(defn image [n] 
  (def im (BufferedImage. 800 200 BufferedImage/TYPE_INT_ARGB))
  (def g (.createGraphics im))
  (def molPrinter (MolPrinter. n))
  (def rect (Rectangle. 20, 20, 360, 200))
  (.setScale molPrinter (* 0.8 (.maxScale molPrinter rect)))
  (.paint molPrinter g rect)
  im)

(image n)

In [ ]:
(def reactor (Reactor.))
(def rxmol (RxnMolecule.))
(def rxmol (MolImporter/importMol "[F,Cl,Br,I:3][C:1]=O.[H:4][O:2][#6]>>[#6][O:2][C:1]=O.[F,Cl,Br,I:3][H:4]"))
(image rxmol)

## KEGG RCLASS

<iframe src="http://www.genome.jp/kegg/reaction/" width = 100% height = 1000></iframe> 

<iframe src="http://www.genome.jp/dbget-bin/www_bget?rc:RC00050" width = 100% height = 1000></iframe> 

## ChemAxon reactor

KEGG Definition:          C1b-C4a:*-*:O1a+R-O4a+R

Reaction:                      C1b-C4a

Difference:                   *-*

Matched:                      O1a+R-O4a+R

In [ ]:
;(smarts "C1b-C4a:-:O1a+R-O4a+R") => "CO>>C=O"
(def RC00050 (MolImporter/importMol "CO>>C=O"))
(image RC00050 )

In [ ]:
(def reactor (Reactor.))
(.setReaction reactor RC00050)
(def ethanol (MolImporter/importMol "ethanol" "name"))
(.setReactants reactor (into-array (list ethanol)))
(def products (.react reactor))
(image (first products))

In [ ]:
(.toFormat (first products) "name:t")

In [ ]:
(def reactor (Reactor.))
(.setReaction reactor RC00050)
(def ethanol (MolImporter/importMol "methanol" "name"))
(.setReactants reactor (into-array (list ethanol)))
(def products (.react reactor))
(image (first products))

In [ ]:
(def furfurol (MolImporter/importMol "furfurol" "name"))
(.setReactants reactor (into-array (list furfurol)))
(def products (.react reactor))
(into-array (map #(.toFormat %1 "smiles") products))
(image (first products))

In [ ]:
(.toFormat (first products) "name:t")

In [ ]:
;(smarts "C1b-C4a:-:O1a+R-O4a+R") => "CO>>C=O"
;(def RC00050 (MolImporter/importMol "[R]C=O>>[R]CO"))
(image RC00050 )

And the reaction can be reversed, which is useful for our future Retrobiosynthesis schemes.

In [ ]:
(def furfural (MolImporter/importMol "furfural" "name"))
(.setReaction reactor RC00050)
(.setReactants reactor (into-array (list furfural)))
(.setReverse reactor true)
(def products (.react reactor))
(image (first products))

In [ ]:
(import com.twosigma.beakerx.jvm.object.OutputContainer)
(import com.twosigma.beakerx.jvm.object.GridOutputContainerLayoutManager)
(def o (OutputContainer.))
(.setLayoutManager o (GridOutputContainerLayoutManager. 2))
(.addItem o (image furfurol))
(.addItem o (image furfural))
o

In [ ]:
(def mcs (MCES.))
(.setMolecules mcs furfurol furfural)
(.search mcs)
(image (.getAsMolecule mcs))

In [ ]:
(require '[clojure.reflect :as r])
(use '[clojure.pprint :only [print-table]])

(print-table (:members (r/reflect mcs)))

In [ ]:
(println (.getUnmatchedTargetAtoms mcs))
(println (.getUnmatchedTargetBonds mcs))
(println (.getUnmatchedQueryAtoms mcs))
(println (.getUnmatchedQueryBonds mcs))

### Calculations

In [ ]:
(def elemental (ElementalAnalyser.))
(.setMolecule elemental furfural)

In [ ]:
(.exactMass elemental);

In [ ]:
(.mass elemental);

In [ ]:
(.massPrecision elemental);

In [ ]:
(.atomCount elemental 8); // oxygen atom count

In [ ]:
(.atomCount elemental 8, 0) ; // non-isotope oxygen count

In [ ]:
(.isotopeFormula elemental)

In [ ]:
(.composition elemental 2)

In [ ]:
(.isotopeComposition elemental 2)

In [ ]:
(.formula elemental)

In [ ]:
(require '[clojure.reflect :as r])
(use '[clojure.pprint :only [print-table]])

(print-table (:members (r/reflect ElementalAnalyser)))

In [ ]:
furfural

In [ ]:
(def gen (ECFPGenerator.))
(def fp1 (ECFP. (ECFPParameters.)))
(.generate gen furfurol fp1)

(def fp2 (ECFP. (ECFPParameters.)))
(.generate gen furfural fp2)

(println "Dissimilarity:" (.getDissimilarity fp1 fp2))
(println "Euclidean:" (.getEuclidean fp1 fp2))
(println "AsymmetricEuclidean:" (.getAsymmetricEuclidean fp1 fp2))
(println "Tanimoto:" (.getTanimoto fp1 fp2))
(println "FP1 Binary:" (.toBinaryString fp1))

Marvin does not support multistep reactions (or pathways).  It can only handle single step reactions.

In [ ]:
(def mapper (AutoMapper.))
(def mol (MolImporter/importMol "C1CCCCC1=O>>C1COC2(CCCCC2)O1"))
(def rm (RxnMolecule/getReaction RC00050 ));
(.map mapper rm )
(image rm)